## Przygotowanie sesji Spark
Zainicjowanie sesji Spark oraz stworzenie schematu bazy danych z której będziemy korzystać.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

Pobranie sesji Spark jest proste dla programisty korzystającego z notatnika, wymaga podania tylko kilku parametrów, ale faktyczna konfiguracja jest bardziej rozbudowana

In [2]:
import os
args = os.environ['PYSPARK_SUBMIT_ARGS'].replace("  ", "\n")
print(args)

--repositories https://oss.sonatype.org/content/repositories/snapshots/
 --jars /tmp/gcs-connector-hadoop2-1.9.17-shaded.jar,/tmp/google-cloud-nio-0.120.0-alpha-shaded.jar
 --conf spark.hadoop.google.cloud.auth.service.account.enable=true
 --conf spark.hadoop.google.cloud.auth.service.account.json.keyfile=/tmp/secrets/ds-lab-sa.json
 --conf spark.kubernetes.driverEnv.GCS_PROJECT_ID=bigdata-datascience
 --conf spark.kubernetes.driverEnv.GOOGLE_APPLICATION_CREDENTIALS=/tmp/secrets/ds-lab-sa.json
 --conf spark.executorEnv.GCS_PROJECT_ID=bigdata-datascience
 --conf spark.executorEnv.GOOGLE_APPLICATION_CREDENTIALS=/tmp/secrets/ds-lab-sa.json
 --conf spark.kubernetes.driver.secrets.ds-lab-sa-secret=/tmp/secrets
 --conf spark.kubernetes.executor.secrets.ds-lab-sa-secret=/tmp/secrets
 --conf spark.hadoop.fs.gs.project.id=bigdata-datascience
 --conf spark.hadoop.fs.gs.impl=com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem
 --conf spark.hadoop.fs.AbstractFileSystem.gs.impl=com.google.cloud.h

Mamy pobraną sesję sparkową. Powstały dodatkowe pody gotowe na realizację obliczeń. A jak zwolnić te zasoby?

In [ ]:
spark.stop()

Ponownie pobieramy sesję Spark. Będziemy z niej korzystać. Po zakończonej pracy należy pamiętać o zastopowaniu sesji.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

## Odczyt danych
Korzystając z sesji Spark można odczytać dane zapisane w lokalizacji dostępnej dla executorów (procesów obliczeniowych) koordynowanych przez Spark. 
Konieczne jest podanie ścieżki dostępowej do pliku i formatu danych (nie jest to jednoznazne z rozszerzeniem pliku).
Dystrybucja Spark udostępnia kilka tzw Data Sources, które odczytują i zapisują dane w określonych formatach (CSV, formaty kolumnowe: parquet/orc).
Data Sources zgodne z opracowanym interfejsem można samodzielnie tworzyć. Na dzisiejszych zajęciach będziemy korzystać z takich zdefiniowanych DS:
* FASTQDataSource
* BAMDataSource
* VCFDataSource

**UWAGA - powyższe sposoby odczytu plików nie są częścią głównej dystrybucji Spark. Wymagana jest dodatkowa konfiguracja.**

In [5]:
import os                               # moduł OS języka Python
user_name = os.environ.get('USER')      # pobieramy zmienną środowiskową USER
bucket = f"gs://edugen-lab-{user_name}2" # konstruujemy sciezke dostepowa do pliku
print(bucket)

gs://edugen-lab-tgambin2


---

`Co oznacza f przed cudzysłowem?`

`Czy jest różnica między stosowaniem apostrofu i cudzysłowu przy definicji zmiennych przechowywujących łańcuchy znaków?`

In [21]:
reads_path = f"{bucket}/fastq/*"  # * oznacza wszystkie pliki we wskazanej lokalizacji. Można podać konkretny plik
fastq_all = spark.read.load(reads_path, format="org.biodatageeks.sequila.datasources.FASTQ.FASTQDataSource")
fastq_all = fastq_all.select("sample_id","seq", "qual")

### Weryfikacja danych

In [22]:
type(fastq_all)      # jaki jest typ danych utworzonej zmiennej? 

pyspark.sql.dataframe.DataFrame

In [23]:
fastq_all.printSchema() # jaki jest schemat danych?

root
 |-- sample_id: string (nullable = true)
 |-- seq: string (nullable = true)
 |-- qual: string (nullable = true)



In [24]:
len(fastq_all.columns)           # wymiary (liczba kolumn)

3

In [25]:
fastq_all.count()               # wymiary (liczba wierszy)

241114

In [26]:
fastq_all.explain(True)              #  plan wykonania

== Parsed Logical Plan ==
'Project [unresolvedalias('sample_id, None), unresolvedalias('seq, None), unresolvedalias('qual, None)]
+- Relation[sample_id#250,instrument_name#251,run_id#252,flowcell_id#253,lane#254,tile#255,pos_x#256,pos_y#257,filter_passed#258,control_num#259,index_seq#260,seq#261,qual#262] org.biodatageeks.sequila.datasources.FASTQ.SequencedFragmentRelation@372e5706

== Analyzed Logical Plan ==
sample_id: string, seq: string, qual: string
Project [sample_id#250, seq#261, qual#262]
+- Relation[sample_id#250,instrument_name#251,run_id#252,flowcell_id#253,lane#254,tile#255,pos_x#256,pos_y#257,filter_passed#258,control_num#259,index_seq#260,seq#261,qual#262] org.biodatageeks.sequila.datasources.FASTQ.SequencedFragmentRelation@372e5706

== Optimized Logical Plan ==
Project [sample_id#250, seq#261, qual#262]
+- Relation[sample_id#250,instrument_name#251,run_id#252,flowcell_id#253,lane#254,tile#255,pos_x#256,pos_y#257,filter_passed#258,control_num#259,index_seq#260,seq#261,qua

DataFrame jest abstrakcją nad innym typem danych (RDD), który jest podstawową rozproszoną strukturą danych. Poprzez DF możemy dostać się do rdd i zweryfikować na przykład liczbę partycji danych.

In [27]:
fastq_all.rdd.getNumPartitions() # liczba partycji (bloków danych)

3

### Podgląd danych

In [28]:
fastq_all.show()

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|      son|GATGCTCAAACTCCAGG...|=>>=@==?>>=>>>?9<...|
|      son|ATTGTGCTTAACAATGC...|995+7))>:,,94=,=9...|
|      son|CATCTATTTAGCTGAAA...|=>:>A?>0=:?=?>;1>...|
|      son|TTCCTGTACCTCCTTCC...|..6=:8,:=>.=+;<>=...|
|      son|CATACCTAGCATGGCCT...|=>>=>??>?>?>>=>?@...|
|      son|ACTAAATTGCCATGGTA...|;:=<>>=<>>=>==<=9...|
|      son|AACACAGGTGGGAATTG...|10<:;=.>>78:>==>>...|
|      son|CATGTTAAGCTGCATGT...|;<<+7===?>>9>=:=<...|
|      son|CTTTAGTAATTTTCCTC...|=?7=?@>>>>===>>?>...|
|      son|TAAATATTTCACAACAC...|;;<=>>:<=>>;>=;<<...|
|      son|ATCTTTGCATTTGACAC...|=>>??><=?>9=;>=?=...|
|      son|AACACCAGCCTGGCCAA...|9<:=:>4)>8><==+5,...|
|      son|AACACTTCTGTAGAAAA...|1><>>@>>@><>?>?1>...|
|      son|GTGAGAAAATAAACCAA...|;9;8>,;==>27=;+83...|
|      son|TATACCTGGGAATGGGG...|==>=>??>>>=>>>>>>...|
|      son|AGGAAATGTCCAGCTTA

In [29]:
fastq_all.show(5)  # pierwsze 5 wierszy

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|      son|GATGCTCAAACTCCAGG...|=>>=@==?>>=>>>?9<...|
|      son|ATTGTGCTTAACAATGC...|995+7))>:,,94=,=9...|
|      son|CATCTATTTAGCTGAAA...|=>:>A?>0=:?=?>;1>...|
|      son|TTCCTGTACCTCCTTCC...|..6=:8,:=>.=+;<>=...|
|      son|CATACCTAGCATGGCCT...|=>>=>??>?>?>>=>?@...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [30]:
fastq_all.show(truncate=False) # bez skracania zawartości kolumn

+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|sample_id|seq                                                                                                                                                    |qual                                                                                                                                                   |
+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+
|son      |GATGCTCAAACTCCAGGTAGTAACCTCAGGTTGTATCATAG

Widok "szerokich" tabel jest nieczytelny, w kolejnych częściach warsztatów zaradzimy temu korzystając z dodatkowej biblioteki.

## Dostęp do wybranych danych

#### Operacja projekcji (SELECT)

In [31]:
fastq_all.select("sample_id").show()

+---------+
|sample_id|
+---------+
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
+---------+
only showing top 20 rows



`Czy operacja select (sample_id) wpłynęła na oryginalny data frame fastq_all?`

In [32]:
fastq_all.printSchema()
fastq_all.show() 

root
 |-- sample_id: string (nullable = true)
 |-- seq: string (nullable = true)
 |-- qual: string (nullable = true)

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|      son|GATGCTCAAACTCCAGG...|=>>=@==?>>=>>>?9<...|
|      son|ATTGTGCTTAACAATGC...|995+7))>:,,94=,=9...|
|      son|CATCTATTTAGCTGAAA...|=>:>A?>0=:?=?>;1>...|
|      son|TTCCTGTACCTCCTTCC...|..6=:8,:=>.=+;<>=...|
|      son|CATACCTAGCATGGCCT...|=>>=>??>?>?>>=>?@...|
|      son|ACTAAATTGCCATGGTA...|;:=<>>=<>>=>==<=9...|
|      son|AACACAGGTGGGAATTG...|10<:;=.>>78:>==>>...|
|      son|CATGTTAAGCTGCATGT...|;<<+7===?>>9>=:=<...|
|      son|CTTTAGTAATTTTCCTC...|=?7=?@>>>>===>>?>...|
|      son|TAAATATTTCACAACAC...|;;<=>>:<=>>;>=;<<...|
|      son|ATCTTTGCATTTGACAC...|=>>??><=?>9=;>=?=...|
|      son|AACACCAGCCTGGCCAA...|9<:=:>4)>8><==+5,...|
|      son|AACACTTCTGTAGAAAA...|1><>>@>>@><>?>?1>...|
|      son|GTGAGAA

Jeśli chcemy zachować wynik działania transformacji (w celu późniejszego wykorzystania) trzeba wynik zachować w zmiennej.

In [33]:
fastq_sample_only = fastq_all.select("sample_id")  

In [34]:
fastq_sample_only.printSchema()
fastq_sample_only.show()

root
 |-- sample_id: string (nullable = true)

+---------+
|sample_id|
+---------+
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
|      son|
+---------+
only showing top 20 rows



In [35]:
fastq_all.select("sample_id","seq").show()

+---------+--------------------+
|sample_id|                 seq|
+---------+--------------------+
|      son|GATGCTCAAACTCCAGG...|
|      son|ATTGTGCTTAACAATGC...|
|      son|CATCTATTTAGCTGAAA...|
|      son|TTCCTGTACCTCCTTCC...|
|      son|CATACCTAGCATGGCCT...|
|      son|ACTAAATTGCCATGGTA...|
|      son|AACACAGGTGGGAATTG...|
|      son|CATGTTAAGCTGCATGT...|
|      son|CTTTAGTAATTTTCCTC...|
|      son|TAAATATTTCACAACAC...|
|      son|ATCTTTGCATTTGACAC...|
|      son|AACACCAGCCTGGCCAA...|
|      son|AACACTTCTGTAGAAAA...|
|      son|GTGAGAAAATAAACCAA...|
|      son|TATACCTGGGAATGGGG...|
|      son|AGGAAATGTCCAGCTTA...|
|      son|GAATCTAATTGTCTTGG...|
|      son|TATAGAGCCCCAACCAC...|
|      son|TTTGAGAAAACTTACAG...|
|      son|ATCTGCTACTATAAGCA...|
+---------+--------------------+
only showing top 20 rows



Powiedzmy, że interesują nas wszystkie kolumny poza qual. Jak to zrobic? Można wylistować wszystkie kolumny poza qual, ale to uciazliwe. Mozna skorzystac z operacji drop.

In [36]:
fastq_no_qual=fastq_all.drop('qual')

`Czy operacja usunięcia kolumny qual wpłynęła na oryginalny data frame fastq_all?`

In [37]:
fastq_no_qual.printSchema()
fastq_no_qual.show()

root
 |-- sample_id: string (nullable = true)
 |-- seq: string (nullable = true)

+---------+--------------------+
|sample_id|                 seq|
+---------+--------------------+
|      son|GATGCTCAAACTCCAGG...|
|      son|ATTGTGCTTAACAATGC...|
|      son|CATCTATTTAGCTGAAA...|
|      son|TTCCTGTACCTCCTTCC...|
|      son|CATACCTAGCATGGCCT...|
|      son|ACTAAATTGCCATGGTA...|
|      son|AACACAGGTGGGAATTG...|
|      son|CATGTTAAGCTGCATGT...|
|      son|CTTTAGTAATTTTCCTC...|
|      son|TAAATATTTCACAACAC...|
|      son|ATCTTTGCATTTGACAC...|
|      son|AACACCAGCCTGGCCAA...|
|      son|AACACTTCTGTAGAAAA...|
|      son|GTGAGAAAATAAACCAA...|
|      son|TATACCTGGGAATGGGG...|
|      son|AGGAAATGTCCAGCTTA...|
|      son|GAATCTAATTGTCTTGG...|
|      son|TATAGAGCCCCAACCAC...|
|      son|TTTGAGAAAACTTACAG...|
|      son|ATCTGCTACTATAAGCA...|
+---------+--------------------+
only showing top 20 rows



## Wartości unikalne
Jeśli chcemy uzyskać unikalne wartości w określonych kolumnach korzystamy z metody distinct().
Operacje na DF można łańcuchowo łączyć, zatem na wyniku działania select() można wywołać kolejne metody.

In [38]:
fastq_all.select('sample_id').distinct().show()

+---------+
|sample_id|
+---------+
|   father|
|   mother|
|      son|
+---------+



In [40]:
fastq_all.count()

241114

# Sortowanie

Do sortowania służy metoda orderBy. Domyślne sortowanie jest rosnące.

In [41]:
fastq_all.select('sample_id').distinct().orderBy('sample_id').show()

+---------+
|sample_id|
+---------+
|   father|
|   mother|
|      son|
+---------+



In [42]:
fastq_all.select('sample_id').distinct().orderBy('sample_id', ascending=False).show()

+---------+
|sample_id|
+---------+
|      son|
|   mother|
|   father|
+---------+



In [43]:
fastq_all.orderBy('sample_id', 'seq', ascending=False).show()  # kierunek sortowania jest wspólny dla listy kolumn

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|      son|TTTTTTTTTTTTTTTTT...|..--61<:::5:+5:<1...|
|      son|TTTTTTTTTTTTTTTTT...|;:;:=++<<=++=<<=+...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<=<<+1:1<<9<2+...|
|      son|TTTTTTTTTTTTTTTTT...|;;--=+<1<2+<:<<=+...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<==<1+:<<<<:<<...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<=+<:<=:511:69...|
|      son|TTTTTTTTTTTTTTTTT...|.:-<61<<56+<<1++<...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<=9+5=,<+<+6<<...|
|      son|TTTTTTTTTTTTTTTTT...|;;;<=<<1+=+1:5+<1...|
|      son|TTTTTTTTTTTTTTTTT...|;:;<==<<==<<<<<<+...|
|      son|TTTTTTTTTTTTTTTTT...|9:-:==:<5:15:<:+9...|
|      son|TTTTTTTTTTTTTTTTT...|;:;<==<<56<<1<<<<...|
|      son|TTTTTTTTTTTTTTTTT...|;:;<==<<<2<<:<<<<...|
|      son|TTTTTTTTTTTTTTTTG...|;:;<=9++5=1+++<<1...|
|      son|TTTTTTTTTTTTTTTTG...|;:8<=+<5+255<1<<8...|
|      son|TTTTTTTTTTTTTTTTC

In [44]:
fastq_all.orderBy('sample_id', ascending=False).orderBy('seq', ascending=True).show() # sortowanie malejace i rosnące na dwóch roznych kolumnach 

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|   mother|AAAAAAAAAAAAAAAAA...|====??=>>>8>>>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|;<9<=>>==:=======...|
|   father|AAAAAAAAAAAAAAAAA...|/;.:/78844<8,=<47...|
|   mother|AAAAAAAAAAAAAAAAA...|====??>>>>>>>>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|==;=??=>=>9>:>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|====??>>>>>>>>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|;;;<=>===9<==7===...|
|   mother|AAAAAAAAAAAAAAAAA...|==;<=?>>=>9>9>==>...|
|   mother|AAAAAAAAAAAAAAAAA...|;;;<=>>==>======<...|
|   mother|AAAAAAAAAAAAAAAAA...|===<??>>>>>>>>>:>...|
|   mother|AAAAAAAAAAAAAAAAA...|===<??>>>>>>=>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|;;;<=>>==>=======...|
|   mother|AAAAAAAAAAAAAAAAA...|====??>=>><>>>>>>...|
|   mother|AAAAAAAAAAAAAAAAA...|;;9:/9;:=.83=####...|
|   mother|AAAAAAAAAAAAAAAAA...|;;;<=>>==>=======...|
|   mother|AAAAAAAAAAAAAAAAA

### Filtrowanie wynikow
Nasz zbiór danych posiada odczyty z 3 próbek. Ograniczmy się do wybranych próbek.

In [45]:
fastq_mother = fastq_all.filter("sample_id = 'mother'")

`Czy już odbył się odczyt danych z fastq?`

In [46]:
fastq_mother.show()

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|   mother|TGGAGTTCTAAATTGTG...|==:=?>=>?>>>>>>>>...|
|   mother|GCATGTGCAAGTATCTT...|;;<<9>>>>9>:=<=?<...|
|   mother|GCCATTAGACGACATTT...|=<=2?<0?=;5>>?>70...|
|   mother|AGGCAGGACTAGTTCCT...|::;,::*-=0:>88-+;...|
|   mother|ATAAGGGGGCCACTTTT...|==8=??>>>?>?>@>==...|
|   mother|TGAAAAACCACATGGAA...|/99;;9=8=>=-2=::3...|
|   mother|GAAATGCTTAATTACCC...|====2??@<=>>==8=0...|
|   mother|CCACAAAAGCCTTTATC...|;;<6<.>=>2+>=8,=9...|
|   mother|CAACTTAACAAATTCAT...|=>;<?>?>>?>=>=>?>...|
|   mother|TTCTCCTAAGTGAAATA...|;:;=9>?==?===<==>...|
|   mother|AGAATTACTGTTTGACT...|</;=:>>>?9/:7;<>4...|
|   mother|AGATATAAATTATATTG...|:.9/;-?.=8+>->-+7...|
|   mother|TTTGAACCTTTACTACA...|====??>>?==?>?=>?...|
|   mother|TCAAGGACTAAGATATA...|;0:<=>>=>?<=9=>=>...|
|   mother|AGTAAAGGTTCTAGTTA...|<=;==?>>=><@7<;9=...|
|   mother|GAGGGTGTCATGGGAAC

In [47]:
fastq_mother.select('sample_id').distinct().show()

+---------+
|sample_id|
+---------+
|   mother|
+---------+



In [48]:
fastq_mother.count()

133160

Warunki można łączyć spójnikami logicznymi. 
Można używać
* operatorów arytmetycznych (=, !=, >, >=, <, <=)
* przynależności do zbioru (IN/NOT IN) 
* porównania znaków (LIKE/NOT LIKE)
* przyrównania do wartości NULL (IS NULL/ IS NOT NULL)

Przy korzystaniu z LIKE można użyć % jako oznaczenie dowolnego ciągu znaków.

Konstrukcja warunku w metodzie filter() jak taka jak w klauzuli WHERE W SQL.

Pokaż odczyty spełniające warunek ze nazwa instrumentu jest pusta, run_id jest >=0 a odczyt zaczyna sie od liter GCA. Pokaz tylko kolumny z filtra oraz nazwe probki

In [49]:
fastq_mother.filter('seq LIKE "GCA%"').select('sample_id',  'seq').show()

+---------+--------------------+
|sample_id|                 seq|
+---------+--------------------+
|   mother|GCATGTGCAAGTATCTT...|
|   mother|GCAGGTACTCATGTTCA...|
|   mother|GCACATCCTGGCCTCCT...|
|   mother|GCAACCTCCACCTCCTA...|
|   mother|GCAGAAGGTGCAAAAGC...|
|   mother|GCATCGAATTTTTTTTC...|
|   mother|GCACTTCCCTGGCCACA...|
|   mother|GCAGTCACCTTCCCAGC...|
|   mother|GCACCTCTGATGCTGGA...|
|   mother|GCAGAAAGCTGGGTGCA...|
|   mother|GCATAATAGACACTAGG...|
|   mother|GCAATCAATCAGATGGC...|
|   mother|GCAACTTCAGTAAAGTC...|
|   mother|GCAAACGAACACAGGAA...|
|   mother|GCATATCATTTCCCATA...|
|   mother|GCATTTTCACTTGCATG...|
|   mother|GCAGTTACAGTCTTAGA...|
|   mother|GCAACTTATAAAATGGG...|
|   mother|GCAACATATAAAATGGG...|
|   mother|GCAACGGACAAGGAGAA...|
+---------+--------------------+
only showing top 20 rows



In [50]:
fastq_mother.select('sample_id', 'seq').filter('seq LIKE "GCA%"').show() # kolejnosc select i filter bez znaczenia

+---------+--------------------+
|sample_id|                 seq|
+---------+--------------------+
|   mother|GCATGTGCAAGTATCTT...|
|   mother|GCAGGTACTCATGTTCA...|
|   mother|GCACATCCTGGCCTCCT...|
|   mother|GCAACCTCCACCTCCTA...|
|   mother|GCAGAAGGTGCAAAAGC...|
|   mother|GCATCGAATTTTTTTTC...|
|   mother|GCACTTCCCTGGCCACA...|
|   mother|GCAGTCACCTTCCCAGC...|
|   mother|GCACCTCTGATGCTGGA...|
|   mother|GCAGAAAGCTGGGTGCA...|
|   mother|GCATAATAGACACTAGG...|
|   mother|GCAATCAATCAGATGGC...|
|   mother|GCAACTTCAGTAAAGTC...|
|   mother|GCAAACGAACACAGGAA...|
|   mother|GCATATCATTTCCCATA...|
|   mother|GCATTTTCACTTGCATG...|
|   mother|GCAGTTACAGTCTTAGA...|
|   mother|GCAACTTATAAAATGGG...|
|   mother|GCAACATATAAAATGGG...|
|   mother|GCAACGGACAAGGAGAA...|
+---------+--------------------+
only showing top 20 rows



<div class="alert alert-block alert-warning">

<b>Zadanie 2_2:</b>Napisz polecenie które policzy ile jest rekordów dla próbki syna które spełniają warunki, że sekwencja odczytu konczy się na TGG a qual zaczyna się od ==. </div>



## Używanie funkcji, kolumny wyliczane

Dostępne są funkcje skalarne (przykład: ROUND, UPPER, CURRENT_DATE) oraz agregujące (MIN, MAX, AVG, SUM, COUNT).
Niektóre funkcje są dostępne "od razu" bez dodatkowych poleceń import. 
Lista funkcji znajduje się : https://spark.apache.org/docs/latest/api/sql/index.html 

In [51]:
fastq_all.selectExpr("*").show()  # pokaż wszystkie kolumny tego DF

+---------+--------------------+--------------------+
|sample_id|                 seq|                qual|
+---------+--------------------+--------------------+
|      son|GATGCTCAAACTCCAGG...|=>>=@==?>>=>>>?9<...|
|      son|ATTGTGCTTAACAATGC...|995+7))>:,,94=,=9...|
|      son|CATCTATTTAGCTGAAA...|=>:>A?>0=:?=?>;1>...|
|      son|TTCCTGTACCTCCTTCC...|..6=:8,:=>.=+;<>=...|
|      son|CATACCTAGCATGGCCT...|=>>=>??>?>?>>=>?@...|
|      son|ACTAAATTGCCATGGTA...|;:=<>>=<>>=>==<=9...|
|      son|AACACAGGTGGGAATTG...|10<:;=.>>78:>==>>...|
|      son|CATGTTAAGCTGCATGT...|;<<+7===?>>9>=:=<...|
|      son|CTTTAGTAATTTTCCTC...|=?7=?@>>>>===>>?>...|
|      son|TAAATATTTCACAACAC...|;;<=>>:<=>>;>=;<<...|
|      son|ATCTTTGCATTTGACAC...|=>>??><=?>9=;>=?=...|
|      son|AACACCAGCCTGGCCAA...|9<:=:>4)>8><==+5,...|
|      son|AACACTTCTGTAGAAAA...|1><>>@>>@><>?>?1>...|
|      son|GTGAGAAAATAAACCAA...|;9;8>,;==>27=;+83...|
|      son|TATACCTGGGAATGGGG...|==>=>??>>>=>>>>>>...|
|      son|AGGAAATGTCCAGCTTA

Dodanie dwóch dodatkowych kolumn wyliczanych 

In [52]:
fastq_all.selectExpr("*", "length(seq) as len_seq", "length(qual) as len_qual" ).show() ## dodanie dwóch kolumn wyliczanych przy uzyciu funkcji LENGTH

+---------+--------------------+--------------------+-------+--------+
|sample_id|                 seq|                qual|len_seq|len_qual|
+---------+--------------------+--------------------+-------+--------+
|      son|GATGCTCAAACTCCAGG...|=>>=@==?>>=>>>?9<...|    151|     151|
|      son|ATTGTGCTTAACAATGC...|995+7))>:,,94=,=9...|    151|     151|
|      son|CATCTATTTAGCTGAAA...|=>:>A?>0=:?=?>;1>...|    151|     151|
|      son|TTCCTGTACCTCCTTCC...|..6=:8,:=>.=+;<>=...|    151|     151|
|      son|CATACCTAGCATGGCCT...|=>>=>??>?>?>>=>?@...|    151|     151|
|      son|ACTAAATTGCCATGGTA...|;:=<>>=<>>=>==<=9...|    151|     151|
|      son|AACACAGGTGGGAATTG...|10<:;=.>>78:>==>>...|    151|     151|
|      son|CATGTTAAGCTGCATGT...|;<<+7===?>>9>=:=<...|    151|     151|
|      son|CTTTAGTAATTTTCCTC...|=?7=?@>>>>===>>?>...|    151|     151|
|      son|TAAATATTTCACAACAC...|;;<=>>:<=>>;>=;<<...|    151|     151|
|      son|ATCTTTGCATTTGACAC...|=>>??><=?>9=;>=?=...|    151|     151|
|     

Alias - nadanie kolumnie lub kolumnie wyliczanej nazwy

In [54]:
extended_fastq = fastq_all.selectExpr("*", "length(seq) as len_s", "length(qual) as len_q" ) # AS alias

In [55]:
extended_fastq.printSchema()

root
 |-- sample_id: string (nullable = true)
 |-- seq: string (nullable = true)
 |-- qual: string (nullable = true)
 |-- len_s: integer (nullable = true)
 |-- len_q: integer (nullable = true)



Dodanie nowej kolumny, dla każdego wiersza zostanie dodana wartość zwracana przez funkcję current_date()

In [56]:
from pyspark.sql.functions import current_date
extended_fastq.withColumn ("date", current_date()).show()

+---------+--------------------+--------------------+-----+-----+----------+
|sample_id|                 seq|                qual|len_s|len_q|      date|
+---------+--------------------+--------------------+-----+-----+----------+
|      son|GATGCTCAAACTCCAGG...|=>>=@==?>>=>>>?9<...|  151|  151|2022-05-13|
|      son|ATTGTGCTTAACAATGC...|995+7))>:,,94=,=9...|  151|  151|2022-05-13|
|      son|CATCTATTTAGCTGAAA...|=>:>A?>0=:?=?>;1>...|  151|  151|2022-05-13|
|      son|TTCCTGTACCTCCTTCC...|..6=:8,:=>.=+;<>=...|  151|  151|2022-05-13|
|      son|CATACCTAGCATGGCCT...|=>>=>??>?>?>>=>?@...|  151|  151|2022-05-13|
|      son|ACTAAATTGCCATGGTA...|;:=<>>=<>>=>==<=9...|  151|  151|2022-05-13|
|      son|AACACAGGTGGGAATTG...|10<:;=.>>78:>==>>...|  151|  151|2022-05-13|
|      son|CATGTTAAGCTGCATGT...|;<<+7===?>>9>=:=<...|  151|  151|2022-05-13|
|      son|CTTTAGTAATTTTCCTC...|=?7=?@>>>>===>>?>...|  151|  151|2022-05-13|
|      son|TAAATATTTCACAACAC...|;;<=>>:<=>>;>=;<<...|  151|  151|2022-05-13|

Dodanie kolumny o stałej wartości dla każdej wartości wymaga wykorzystania funkcji lit (), która przekształci stała wartość w kolumnę.

In [57]:
from pyspark.sql.functions import lit

extended_fastq.withColumn("imported_by", lit(user_name)).withColumn("format", lit('FASTQ')).show()

+---------+--------------------+--------------------+-----+-----+-----------+------+
|sample_id|                 seq|                qual|len_s|len_q|imported_by|format|
+---------+--------------------+--------------------+-----+-----+-----------+------+
|      son|GATGCTCAAACTCCAGG...|=>>=@==?>>=>>>?9<...|  151|  151|    tgambin| FASTQ|
|      son|ATTGTGCTTAACAATGC...|995+7))>:,,94=,=9...|  151|  151|    tgambin| FASTQ|
|      son|CATCTATTTAGCTGAAA...|=>:>A?>0=:?=?>;1>...|  151|  151|    tgambin| FASTQ|
|      son|TTCCTGTACCTCCTTCC...|..6=:8,:=>.=+;<>=...|  151|  151|    tgambin| FASTQ|
|      son|CATACCTAGCATGGCCT...|=>>=>??>?>?>>=>?@...|  151|  151|    tgambin| FASTQ|
|      son|ACTAAATTGCCATGGTA...|;:=<>>=<>>=>==<=9...|  151|  151|    tgambin| FASTQ|
|      son|AACACAGGTGGGAATTG...|10<:;=.>>78:>==>>...|  151|  151|    tgambin| FASTQ|
|      son|CATGTTAAGCTGCATGT...|;<<+7===?>>9>=:=<...|  151|  151|    tgambin| FASTQ|
|      son|CTTTAGTAATTTTCCTC...|=?7=?@>>>>===>>?>...|  151|  151|

<div class="alert alert-block alert-warning">

<b>Zadanie 2_3:</b>

Napisz polecenie które stworzy ramkę danych zawierającą sklejenie wartości dwóch kolumn (sample_id) oraz daty eksperymentu (dodaj kolumne z wartościami 2019-01-15) . W wynikach chcemy mieć tylko dane matki i ojca. Kolumny wynikowe: nazwa próbki, seq, qual, data eksperymentu oraz scalona nazwa probki oraz data eksperymenty (np father-2019-01-15). Posortuj po nazwie próbki. Pokaż schemat ramki. Upewnij się, że data eksperymentu jest typu date. 

* Zwróć uwagę na potrzebę konwersji ciągu znaków na datę
</div>


### Instrukcje warunkowe przy kolumnach wyliczanych

In [58]:
fastq_dates=fastq_all.selectExpr('*', 'if(sample_id = "son",to_date("2018-11-10"), to_date("2019-01-15")) as experiment_date')

<div class="alert alert-block alert-warning">
<b>Zadanie 2_4:</b> Napisz polecenia, które zweryfikuje czy daty eksperymentów zostały dodane poprawnie </div>


## Grupowanie

In [61]:
fastq_all.groupBy("sample_id").show()

AttributeError: 'GroupedData' object has no attribute 'show'

In [62]:
type(fastq_all.groupBy("sample_id")) # to nie jest DF

pyspark.sql.group.GroupedData

In [63]:
sample_count=fastq_all.groupBy("sample_id").count()

In [64]:
type(sample_count)

pyspark.sql.dataframe.DataFrame

In [65]:
sample_count.show()

+---------+------+
|sample_id| count|
+---------+------+
|   father| 53770|
|   mother|133160|
|      son| 54184|
+---------+------+



In [66]:
sample_count.orderBy("count").show()

+---------+------+
|sample_id| count|
+---------+------+
|   father| 53770|
|      son| 54184|
|   mother|133160|
+---------+------+



<div class="alert alert-block alert-warning">
<b>Zadanie 2_5:</b> Napisz funkcje, ktora znajdzie rozklady jakosci dla 1, 2, 3 i 4 pozycji odczytu.
*Nastepnie zaprezentuj wyniki w postaci serii histogramów </div>

### Mapowanie do genomu referencyjnego

UWAGA - na dzisiejszych zajęciach ten kod nie będzie uruchamiany

Wykonamy mapowanie do genomu referencyjnych korzystając z rozproszenia danych miedzy procesy obliczeniowe sparka.

Przygotowanie ścieżek do plików.

In [ ]:
#import os
#user_name = os.environ.get('USER')
#bucket = f"gs://edugen-lab-{user_name}"

#reads_file_path = f"{bucket}/fastq/mother.fastq"
#ref_path = "/mnt/data/mapping/ref/ref.fasta"

Konstruujemy komendę, która będzie uruchamiana na procesach obliczeniowych. Potrzebne narzędzia muszą być dostępne na węzłach obliczeniowych.

In [ ]:
#command = f'bwa mem -p {ref_path} - | samtools fixmate -m - - | samtools sort  | samtools markdup -r -S - -  | samtools addreplacerg  -r "ID:S1" -r "SM:S1"  -r "PL:ILLUMINA" - | samtools view -b -'

Żeby wykonać rozproszone obliczenia na danych genomicznych nalezy wykorzystac dodatkową bibliotekę.

In [ ]:
#from pyseqtender import SeqTenderAlignment

#seq_aligner = SeqTenderAlignment(spark, reads_file_path, command)
#alignments_rdd = seq_aligner.pipe_reads()

Zapisujemy plik na kubełek.

In [ ]:
#bam_file_path = f"{bucket}/bam/mother10.bam"
#seq_aligner.save_reads(bam_file_path, alignments_rdd)

In [ ]:
#!gsutil ls gs://edugen-lab-$USER/bam

<div class="alert alert-block alert-warning">
<b>Zadanie 2_6:</b> Na podstawie notatników z zajęć z genomiki wyświetl fragment pliku BAM w widge'cie IGV.  </div>



Kończymy notatniki, należy zamknąć sesję.

In [67]:
spark.stop()